In [8]:
###### INCLUSION CRITERIA ANALYSIS###
####The below code 
#Creators: Xiaoru Dong
#Contributors:
#Last updated: 12/06/2018

In [ ]:
import pandas as pd
from collections import OrderedDict

In [9]:
# Assign spreadsheet filename to `file`
file = '/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/Annotations_All.xlsx'

# Load spreadsheet
xl = pd.ExcelFile(file)

# Load a sheet into a DataFrame by name: df1
df = xl.parse('All Reviews (5420)')

In [10]:
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", 
              "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 
              'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 
              'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 
              'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 
              'the', 'if', 'because', 'as', 'until', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 'into',
              'through', 'during', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 
              'under', 'again', 'further', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'few', 'more', 
              'most', 'some', 'such', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 
              'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain']

In [4]:
df

,Inclusion Criteria,RCT
0,The trials in the review include only those in...,NaN
1,Randomised controlled trials (RCTs) or quasi-R...,NaN
2,Only randomized controlled trials of at least ...,x
3,We sought to identify all randomised unconfoun...,x
4,We included all relevant randomised controlled...,x
5,For efficacy we examined randomized trials com...,NaN
6,All acceptable randomized trials which address...,NaN
7,All randomised and quasi-randomised studies co...,NaN
8,Trials utilising random or quasi-random patien...,NaN
9,Only randomised controlled trials were conside...,x


In [11]:
lineNum = len(df)

In [5]:
results = []
wordlist = []
rct = []
eachline = []

# split the sentence into words and save in a list
for i in range(lineNum): 
    
    string_rct = str(df['Only RCTs'][i])
    if string_rct == 'nan':
        string_rct = '0.0'
    if string_rct == 'x':
        string_rct = '1.0'
    rct.append(string_rct)
    
    string = str(df['Inclusion Criteria'][i]).lower().strip()
    wordlist = string.split(' ')
    
    for j in range(len(wordlist)):
        wordlist[j] = wordlist[j].replace('.', '')
        wordlist[j] = wordlist[j].replace(')', '')
        wordlist[j] = wordlist[j].replace('(', '')
        wordlist[j] = wordlist[j].replace(',', '')
        wordlist[j] = wordlist[j].replace("'", '')
        wordlist[j] = wordlist[j].replace('"', '')
        results.append(wordlist[j])
    eachline.append(wordlist)

In [6]:
results = list(OrderedDict.fromkeys(results))
results = [x for x in results if x] #remove empty element in the list
len(results)

7292

In [7]:
# remove stop words
results = [word for word in results if word not in stop_words]
len(results)

7202

In [23]:
# output features
with open("/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/word_bag_0.txt","w") as fp: 
    for word in results:
        fp.write(word + "\n")

In [24]:
# get arff file used as the input in Weka
with open("/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/classification_fall_weka_input_0.arff","w") as fp:        #this line of code to create a new file
    fp.write('''@RELATION classification_analysis\n''')
    
    for i in range(len(results)): 
        fp.write('''@ATTRIBUTE ''' + "word" + str(i) + ''' NUMERIC\n''') 
    
    fp.write('''@ATTRIBUTE class {1.0, 0.0}\n''')
    fp.write('''@DATA\n''')

    #this code is to check if a word appear in "topword_to_id" list or not. If yes, then write "1", if no then write "0"
    
    for i in range(lineNum): 
        binary = {}
        for word in results:
            if word in eachline[i]:
                binary[word] = 1
            else:
                binary[word] = 0
            fp.write(str(binary[word]) + ",")
        fp.write(rct[i] + "\n")

In [ ]:
# infomative words without removing same meaning words

In [8]:
# read the informative words and save them in a list
info = xl.parse('Informative Words (1662)')
info_wordlist = []
for word in info['Word Code']:
    info_wordlist.append(results[word])

In [10]:
len(info_wordlist)

1662

In [11]:
# output features
with open("/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/InfoWord/informative_words_0.txt","w") as fp:
    for i in range(len(info_wordlist)):
        fp.write(info_wordlist[i] + "\n")

In [12]:
# get arff file used as the input in Weka
with open("/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/InfoWord/classification_fall_weka_input_info_0.arff","w") as fp:        #this line of code to create a new file
    fp.write('''@RELATION classification_analysis\n''')
    
    for i in range(len(info_wordlist)): 
        fp.write('''@ATTRIBUTE ''' + "infoword" + str(i) + ''' NUMERIC\n''') 
    
    fp.write('''@ATTRIBUTE class {1.0, 0.0}\n''')
    fp.write('''@DATA\n''')

    #this code is to check if a word appear in "topword_to_id" list or not. If yes, then write "1", if no then write "0"
    
    for i in range(lineNum):
        binary = {}
        for word in info_wordlist:
            if word in eachline[i]:
                binary[word] = 1
            else:
                binary[word] = 0
            fp.write(str(binary[word]) + ",")
        fp.write(rct[i] + "\n")

In [ ]:
# remove same meaning words

In [12]:
results = []
wordlist = []
rct = []
eachline = []

#replace the same meaning words
quasi = ['quasi-rcts', 'quasi-randomized', 'quasi-randomised', 'quasi-random', 'quasi-randomly']
cct = ['cct', 'ccts']
cba = ['cba', 'cbas', 'before-after']
privative = ["not","didn't", "don't", "won't", "isn't", "aren't", "wasn't", "weren't", "needn't", "wouldn't", "shouldn't", "can't", "couldn't", "no"]

# split the sentence into words and save in a list
for i in range(lineNum): 
    
    string_rct = str(df['Only RCTs'][i])
    if string_rct == 'nan':
        string_rct = '0.0'
    if string_rct == 'x':
        string_rct = '1.0'
    rct.append(string_rct)
    
    string = str(df['Inclusion Criteria'][i]).lower().strip()
    wordlist = string.split(' ')
    
    for j in range(len(wordlist)):
        wordlist[j] = wordlist[j].replace('.', '')
        wordlist[j] = wordlist[j].replace(')', '')
        wordlist[j] = wordlist[j].replace('(', '')
        wordlist[j] = wordlist[j].replace(',', '')
        wordlist[j] = wordlist[j].replace("'", '')
        wordlist[j] = wordlist[j].replace('"', '')
        if wordlist[j] in quasi:
            wordlist[j] = 'quasi-rcts'
        if wordlist[j] in cct:
            wordlist[j] = 'cct'
        if wordlist[j] in cba:
            wordlist[j] = 'cba'
        if wordlist[j] in privative:
            wordlist[j] = 'not'
        results.append(wordlist[j])
    eachline.append(wordlist)

In [13]:
results = list(OrderedDict.fromkeys(results))
results = [x for x in results if x] #remove empty element in the list
results = [word for word in results if word not in stop_words]
len(results)

7194

In [37]:
# output features
with open("/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/word_bag.txt","w") as fp: 
    for word in results:
        fp.write(word + "\n")

In [38]:
# get arff file used as the input in Weka
with open("/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/classification_fall_weka_input.arff","w") as fp:        #this line of code to create a new file
    fp.write('''@RELATION classification_analysis\n''')
    
    for i in range(len(results)): 
        fp.write('''@ATTRIBUTE ''' + "word" + str(i) + ''' NUMERIC\n''') 
    
    fp.write('''@ATTRIBUTE class {1.0, 0.0}\n''')
    fp.write('''@DATA\n''')

    #this code is to check if a word appear in "topword_to_id" list or not. If yes, then write "1", if no then write "0"
    
    for i in range(lineNum): 
        binary = {}
        for word in results:
            if word in eachline[i]:
                binary[word] = 1
            else:
                binary[word] = 0
            fp.write(str(binary[word]) + ",")
        fp.write(rct[i] + "\n")

In [14]:
# infomative words 

In [14]:
# read the informative words and save them in a list
info = xl.parse('Informative Words2 (1656)')
info_wordlist = []
for word in info['Word Code']:
    info_wordlist.append(results[word])

,Word Code,Informative Word
0,23,quasi-rcts
1,60,birth
2,168,alternation
3,72,and
4,59,date
5,484,methods
6,509,cct
7,49,allocation
8,15,or
9,2470,predictable


In [16]:
len(info_wordlist)

1656

In [11]:
# output features
with open("/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/InfoWord/informative_words.txt","w") as fp:
    for i in range(len(info_wordlist)):
        fp.write(info_wordlist[i] + "\n")

In [12]:
# get arff file used as the input in Weka
with open("/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/InfoWord/classification_fall_weka_input_info.arff","w") as fp:        #this line of code to create a new file
    fp.write('''@RELATION classification_analysis\n''')
    
    for i in range(len(info_wordlist)): 
        fp.write('''@ATTRIBUTE ''' + "infoword" + str(i) + ''' NUMERIC\n''') 
    
    fp.write('''@ATTRIBUTE class {1.0, 0.0}\n''')
    fp.write('''@DATA\n''')

    #this code is to check if a word appear in "topword_to_id" list or not. If yes, then write "1", if no then write "0"
    
    for i in range(lineNum):
        binary = {}
        for word in info_wordlist:
            if word in eachline[i]:
                binary[word] = 1
            else:
                binary[word] = 0
            fp.write(str(binary[word]) + ",")
        fp.write(rct[i] + "\n")

In [ ]:
# reselct informative words

In [17]:
# read the informative words and save them in a list
info2 = xl.parse('Most Informative Words (584)')
info2_wordlist = []
for word in info2['Word Code']:
    info2_wordlist.append(info_wordlist[word])

,Word Code,Word Root,domain-specific words,numbers and signs,words informative to us,Linh's comment,Jodi's comment
0,0,quasi-rcts,NaN,NaN,x,NaN,NaN
1,1,birth,X,NaN,x,NaN,NaN
2,2,alternation,NaN,NaN,x,NaN,this is NOT categorized -- what will you do wi...
3,3,and,NaN,NaN,x,NaN,NaN
4,4,date,NaN,NaN,x,NaN,this is NOT categorized -- what will you do wi...
5,5,methods,NaN,NaN,x,NaN,NaN
6,6,cct,NaN,NaN,x,NaN,NaN
7,7,allocation,NaN,NaN,x,NaN,NaN
8,8,or,NaN,NaN,x,NaN,NaN
9,9,predictable,NaN,NaN,x,NaN,this is NOT categorized -- what will you do wi...


In [19]:
len(info2_wordlist)

584

In [14]:
# output features
with open("/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/InfoWord/informative2_words.txt","w") as fp:
    for i in range(len(info2_wordlist)):
        fp.write(info2_wordlist[i] + "\n")

In [15]:
# get arff file used as the input in Weka
with open("/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/InfoWord/classification_fall_weka_input_info2.arff","w") as fp:        #this line of code to create a new file
    fp.write('''@RELATION classification_analysis\n''')
    
    for i in range(len(info2_wordlist)): 
        fp.write('''@ATTRIBUTE ''' + "info2word" + str(i) + ''' NUMERIC\n''') 
    
    fp.write('''@ATTRIBUTE class {1.0, 0.0}\n''')
    fp.write('''@DATA\n''')

    #this code is to check if a word appear in "topword_to_id" list or not. If yes, then write "1", if no then write "0"
    
    for i in range(lineNum):
        binary = {}
        for word in info2_wordlist:
            if word in eachline[i]:
                binary[word] = 1
            else:
                binary[word] = 0
            fp.write(str(binary[word]) + ",")
        fp.write(rct[i] + "\n")

In [ ]:
# Manual clean the informative words

In [20]:
# read the informative words after manual analysis
info3_wordlist = []
for i in range(len(info2_wordlist)):
    if info2['Words Informative to Us'][i] == "x":
        info3_wordlist.append(info2_wordlist[i])

In [23]:
len(info3_wordlist)

407

In [21]:
# output features
with open("/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/InfoWord/informative3_words.txt","w") as fp:
    for i in range(len(info3_wordlist)):
        fp.write(info3_wordlist[i] + "\n")

In [22]:
# get arff file used as the input in Weka
with open("/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/InfoWord/classification_fall_weka_input_info3.arff","w") as fp:        #this line of code to create a new file
    fp.write('''@RELATION classification_analysis\n''')
    
    for i in range(len(info3_wordlist)): 
        fp.write('''@ATTRIBUTE ''' + "info3word" + str(i) + ''' NUMERIC\n''') 
    
    fp.write('''@ATTRIBUTE class {1.0, 0.0}\n''')
    fp.write('''@DATA\n''')

    #this code is to check if a word appear in "topword_to_id" list or not. If yes, then write "1", if no then write "0"
    
    for i in range(lineNum):
        binary = {}
        for word in info3_wordlist:
            if word in eachline[i]:
                binary[word] = 1
            else:
                binary[word] = 0
            fp.write(str(binary[word]) + ",")
        fp.write(rct[i] + "\n")